In [1]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
"""
notes:

cant find play direction fot cfb
can do sportsradar to get hash & play direction

can do nflsavant for play direction

try play direction with just nflsavant

reach out to ian & other folks
"""

# nflsavant

In [50]:
files = []
for file in os.listdir('../data'):
    if 'nfl' in file:
        print(file)
        files.append(
            pd.read_csv(f'../data/{file}', header = 0, usecols = range(45), low_memory=False)
        )

nflsavant_2019.csv
nflsavant_2018.csv
nflsavant_2020.csv
nflsavant_2013.csv
nflsavant_2015.csv
nflsavant_2014.csv
nflsavant_2016.csv
nflsavant_2017.csv


In [69]:
# files[1].head(3).iloc[:,40:60]

In [52]:
def savant_clean(file_list):
    
    # I don't have boundary which sucks
    
    cleaned_files = []
    
    for file in file_list:
        # 2013 has sporatic miss-inputs
        # Change to numeric & turn coerce strings to NA
        file['PASS'] = pd.to_numeric(file.IsPass, errors = 'coerce')
        file['RUN'] = pd.to_numeric(file.IsRush, errors = 'coerce')
         
        
        # Run Clean
        file['RUN_LEFT'] = file.RushDirection.str.contains('LEFT').replace({True:1, False:0})
        file['RUN_RIGHT'] = file.RushDirection.str.contains('RIGHT').replace({True:1, False:0})
        file['RUN_MIDDLE'] = file.RushDirection.str.contains('CENTER').replace({True:1, False:0})
        file['RUN_DIR'] = np.where(file['RUN_LEFT'] == 1, 0,
                                    np.where(file['RUN_MIDDLE'] == 1, 1,
                                              np.where(file['RUN_RIGHT'] == 1, 2,
                                                       -99)
                                            )
                                   )
        # Pass Clean
        file['PASS_SHORT'] = file.PassType.str.contains('SHORT').replace({True:1, False:0})
        file['PASS_DEEP'] = file.PassType.str.contains('DEEP').replace({True:1, False:0})

        file['PASS_LEFT'] = file.PassType.str.contains('LEFT').replace({True:1, False:0})
        file['PASS_RIGHT'] = file.PassType.str.contains('RIGHT').replace({True:1, False:0})
        file['PASS_MIDDLE'] = file.PassType.str.contains('MIDDLE').replace({True:1, False:0})
        
        file['PASS_DIR'] = np.where(file['PASS_LEFT'] == 1, 0,
                                    np.where(file['PASS_RIGHT'] == 1, 1,
                                              np.where(file['PASS_MIDDLE'] == 1, 2,
                                                       -99)
                                            )
                                   )

        # Formation Clean
        file['FORMATION_SHOTGUN'] = file.Formation.str.contains('SHOTGUN').replace({True:1, False:0})
        file['FORMATION_UNDER_CENTER'] = file.Formation.str.contains('CENTER').replace({True:1, False:0})
        
        # Date Clean
        file['MONTH'] = pd.to_datetime(file['GameDate']).dt.month
        file['DAY'] = pd.to_datetime(file['GameDate']).dt.day
        
        # Yardline Clean
        file['YARD'] = np.where(
            
            file['YardLineDirection'] == 'OWN', 
            file['YardLineFixed'] * -1, 
            
            np.where(file['YardLineDirection'] == 'OPP',
                     file['YardLineFixed'], 
                     -99
                    )
        )
        cleaned_file = file[
            [
                'GameId',
                'OffenseTeam','DefenseTeam',
                'Quarter', 'Minute','Second',
                'YARD', 'Down','ToGo',
                'SeasonYear','MONTH','DAY',
                'PASS','RUN',
                'RUN_DIR','RUN_LEFT', 'RUN_RIGHT', 'RUN_MIDDLE',
                'PASS_DIR', 'PASS_LEFT', 'PASS_RIGHT', 'PASS_MIDDLE',
                'PASS_SHORT','PASS_DEEP',
                'FORMATION_SHOTGUN','FORMATION_UNDER_CENTER',
                'Description'
            ]
        ]
        cleaned_files.append(cleaned_file)
    
    big_file = pd.concat(cleaned_files)
    big_file = big_file.query('(PASS == 1) | (RUN == 1)').reset_index(drop=True).sort_values('SeasonYear')
    return big_file

In [53]:
nfl_data = savant_clean(files)

In [54]:
nfl_data.columns

Index(['GameId', 'OffenseTeam', 'DefenseTeam', 'Quarter', 'Minute', 'Second',
       'YARD', 'Down', 'ToGo', 'SeasonYear', 'MONTH', 'DAY', 'PASS', 'RUN',
       'RUN_DIR', 'RUN_LEFT', 'RUN_RIGHT', 'RUN_MIDDLE', 'PASS_DIR',
       'PASS_LEFT', 'PASS_RIGHT', 'PASS_MIDDLE', 'PASS_SHORT', 'PASS_DEEP',
       'FORMATION_SHOTGUN', 'FORMATION_UNDER_CENTER', 'Description'],
      dtype='object')

In [58]:
nfl_data.head(20)['Description'][97874]

'(3:22) (SHOTGUN) 10-R.GRIFFIN PASS INCOMPLETE SHORT LEFT TO 88-P.GARCON.'

# nflfastR

In [6]:
# this is data from 

# need to assess the level of missing for the variables I desire
#Enter desired years of data
YEARS = [2017]

data = pd.DataFrame()

for i in YEARS:  
    #low_memory=False eliminates a warning
    i_data = pd.read_csv('https://github.com/nflverse/nflfastR-data/blob/master/data/' \
                         'play_by_play_' + str(i) + '.csv.gz?raw=True',
                         compression='gzip', low_memory=False)

    #sort=True eliminates a warning and alphabetically sorts columns
    data = data.append(i_data, sort=True)

#Give each row a unique index
data.reset_index(drop=True, inplace=True)

In [56]:

list(data)

# string with team abbr
data['side_of_field']

# has Team abbr then yardline
# combine with possession team for +/- yard
# data['split_loc'] = data['yrdln'].str.find(' ')

# data['yrdln'].str.find(' ').values

# Nope looks like side of field is equal to the 
subset = data.dropna(subset = ['yrdln','side_of_field'])
subset.loc[~(subset['yrdln'].str.split().str[0] == subset['side_of_field'])][['side_of_field','yrdln']].query('side_of_field != "50"')

# data['yrdln'].apply(lambda row: row.split())

# run vars
# run_location	String indicator for location of run: left, middle, or right.
# run_gap	String indicator for line gap of run: end, guard, or tackle


# pass_length	String indicator for pass length: short or deep.
# pass_location	String indicator for pass location: left, middle, or right.

,side_of_field,yrdln


# sports data stuff 

In [25]:
import zipfile
from zipfile import io
# archive = zipfile.ZipFile('images.zip', 'r')
# imgdata = archive.read('img_01.png')

In [26]:
archive = zipfile.ZipFile('../data/CFB_PBP_Data.zip', 'r')
# archive = archive.read('CFB_PBP_Data.zip')
# bytes_io = io.BytesIO(archive)
dir(archive)
archive.filelist
archive.extract('CFB_PBP_Data/Data Dictionary.txt', path ='../data/')

'../data/CFB_PBP_Data/Data Dictionary.txt'

In [67]:
out =open('../data/CFB_PBP_Data/2014/2014_CFB_Data.csv', 'r', encoding="ISO-8859-1")
# out =open('../data/CFB_PBP_Data/2014/2014_CFB_Data.txt', 'r')

In [68]:
out

<_io.TextIOWrapper name='../data/CFB_PBP_Data/2014/2014_CFB_Data.csv' mode='r' encoding='ISO-8859-1'>

In [69]:
out = pd.read_csv(out)

In [71]:
list(out)
out['playstring']

0         Hayden Lekacz kickoff for 64 yds , Khalif Herb...
1         P.J. Walker pass complete to Jalen Fitzpatrick...
2               Kenneth Harper run for 5 yds to the Temp 22
3         P.J. Walker pass complete to Jalen Fitzpatrick...
4         P.J. Walker pass complete to Nate Hairston for...
                                ...                        
142968     Ezekiel Elliott 1 Yd Run (Sean Nuernberger Kick)
142969    Kyle Clinton kickoff for 46 yds , Charles Nels...
142970    Marcus Mariota pass complete to Byron Marshall...
142971        Marcus Mariota pass incomplete to Keanon Lowe
142972    Marcus Mariota pass intercepted Eli Apple retu...
Name: playstring, Length: 142973, dtype: object

# cfbfastR

In [50]:

import cfbfastR

cfb_df = cfbfastR.cfb.load_cfb_pbp(seasons=range(2011,2013))
print(cfb_df.head())


   home_score  scoring_play  priority  stat_yardage  away_score  \
0         0.0         False     False             7         0.0   
1         0.0         False     False             5         0.0   
2         0.0         False     False             4         0.0   
3         0.0         False     False             1         0.0   
4         0.0         False     False             0         0.0   

            modified            id  \
0  2011-09-01T23:18Z  3.124400e+11   
1  2011-09-01T23:18Z  3.124400e+11   
2  2011-09-01T23:18Z  3.124400e+11   
3  2011-09-01T23:18Z  3.124400e+11   
4  2011-09-01T23:19Z  3.124400e+11   

                                                text  period_number  \
0  Randy Wright pass complete to Anthony Soto for...              1   
1  Josh Reese rush for 5 yards to the UCDav 36, t...              1   
2  Josh Reese rush for 4 yards to the UCDav 40, t...              1   
3  Josh Reese rush for 1 yard to the UCDav 41, ta...              1   
4  Randy Wrig

In [65]:
cfb_df['middle_8'].sum()

#  'short_rush_attempt',
#  'power_rush_success',
#  'power_rush_attempt',
#  'standard_down',
#  'passing_down',
#  'tfl',
#  'tfl_pass',
#  'tfl_rush',
#  'havoc',
#  'havoc_pass',
#  'havoc_rush',

50084

In [52]:
list(cfb_df)

['home_score',
 'scoring_play',
 'priority',
 'stat_yardage',
 'away_score',
 'modified',
 'id',
 'text',
 'period_number',
 'start_short_down_distance_text',
 'start_possession_text',
 'start_down_distance_text',
 'start_distance',
 'start_yard_line',
 'start_team_id',
 'start_down',
 'start_yards_to_endzone',
 'clock_display_value',
 'type_id',
 'type_text',
 'type_abbreviation',
 'end_short_down_distance_text',
 'end_possession_text',
 'end_down_distance_text',
 'end_distance',
 'end_yard_line',
 'end_team_id',
 'end_down',
 'end_yards_to_endzone',
 'scoring_type_display_name',
 'scoring_type_name',
 'scoring_type_abbreviation',
 'drive_id',
 'drive_display_result',
 'drive_is_score',
 'drive_team_short_display_name',
 'drive_team_display_name',
 'drive_team_name',
 'drive_team_abbreviation',
 'drive_yards',
 'drive_offensive_plays',
 'drive_result',
 'drive_description',
 'drive_short_display_result',
 'drive_time_elapsed_display_value',
 'drive_start_period_number',
 'drive_start_

# cfb stats

In [49]:
archive = zipfile.ZipFile('../data/cfbstats.com-2005-1.5.0.zip', 'r')
# archive = archive.read('CFB_PBP_Data.zip')
# bytes_io = io.BytesIO(archive)
dir(archive)
archive.filelist
archive.extract('drive.csv', path ='../data/')

'../data/drive.csv'

In [42]:
out = open('../data/drive.csv', 'r')

In [44]:
file = pd.read_csv(out)

In [47]:
file

,Game Code,Drive Number,Team Code,Start Period,Start Clock,Start Spot,Start Reason,End Period,End Clock,End Spot,End Reason,Plays,Yards,Time Of Possession,Red Zone Attempt
0,690002820050901,1,28,1,900.0,80,KICKOFF,1,826.0,89,PUNT,3,-9,74.0,0
1,690002820050901,2,690,1,826.0,54,PUNT,1,655.0,46,PUNT,6,8,171.0,0
2,690002820050901,3,28,1,655.0,81,PUNT,1,356.0,0,TOUCHDOWN,14,81,299.0,1
3,690002820050901,4,690,1,356.0,80,KICKOFF,1,166.0,43,PUNT,9,37,190.0,0
4,690002820050901,5,28,1,166.0,85,PUNT,1,13.0,0,TOUCHDOWN,7,85,153.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18871,703065720060104,21,657,4,526.0,80,KICKOFF,4,402.0,0,TOUCHDOWN,4,80,124.0,0
18872,703065720060104,22,703,4,402.0,69,KICKOFF,4,243.0,0,TOUCHDOWN,8,69,159.0,1
18873,703065720060104,23,657,4,243.0,66,KICKOFF,4,129.0,44,DOWNS,6,22,114.0,0
18874,703065720060104,24,703,4,129.0,56,DOWNS,4,19.0,0,TOUCHDOWN,10,56,110.0,1


# sportsradar

In [2]:
import requests
import pandas as pd
import joblib
import time
import os
import collections

## NFL

In [18]:
def flatten(d, parent_key="", sep="_"):
    items = []
    for k, v in d.items():
        new_key = parent_key + sep + k if parent_key else k
        if isinstance(v, collections.MutableMapping):
            items.extend(flatten(v, new_key, sep=sep).items())
        else:
            items.append((new_key, v))
    return dict(items)


# this could be done better with recursion 
def flat_pbp(pbp):
    new = []
    for quarter in range(len(pbp['periods'].copy())):
        q_doc = pbp['periods'][quarter].copy()

        for drive in range(len(q_doc['pbp'])):
            drive_doc = q_doc['pbp'][drive].copy()

            if drive_doc['type'] == 'drive':

                for play in range(len(drive_doc['events'])):
                    play_doc = drive_doc['events'][play].copy()

                    if play_doc['type'] == 'play':

                        if play_doc['play_type'] in ['rush','pass']:

#                             play_doc.update(
#                                 {
#                                     'details': [flatten(doc) for doc in play_doc['details']][0]
#                                 }
#                             )

                            play_doc.update(
                                {
                                    'home_team': pbp['summary']['home']['alias'],
                                    'away_team': pbp['summary']['away']['alias'],
                                    'weather': pbp['weather'],
                                    'quarter': q_doc['number'],
                                    'drive_num': drive_doc['sequence']
                                }
                            )
                            new.append(play_doc)
    return pd.DataFrame([flatten(doc) for doc in new])
                            
def clean_pbp(df):
    df = df.rename(columns = 
               {
                   'start_situation_down': 'down',
                   'start_situation_location_yardline': 'yard_line',
                   'start_situation_location_alias': 'posession_team',
                   'start_situation_yfd': 'yards_to_first_down',
                   
#                    'details_direction': 'play_dir',
#                    'details_start_location_alias': 'posession_team',
#                    'details_start_location_yardline': 'yardline',
               }
              )

    df = df[
        [
            'created_at',
            'home_team',
            'away_team',
            'home_points',
            'away_points',
            'clock',
            'quarter',
            'posession_team',
            'down',
            'hash_mark',
            'yard_line',
            'yards_to_first_down',
            'play_type',
            'play_direction',
            'left_tightends',
            'right_tightends',
            'qb_at_snap',
            'description',
        ]
    ]
    return df

    

In [18]:
os.listdir('../data/sportsradar/pbp_json/2020')

_2016_1 = joblib.load('../data/sportsradar/pbp_json/2016/pbp_2016_002038da-7d7f-4c8b-97fe-8487804d1f80_.pkl')

_2020_1 = joblib.load('../data/sportsradar/pbp_json/2020/pbp_2020_c4a0981e-cb75-4c5a-8310-31e0038f8402_.pkl')

# don't need details
# only need description
# can get left guard, right tackle, etc from description
# can get short or long for passes too
df = clean_pbp(flat_pbp(_2020_1))

In [28]:
access_level = 'trial'
version = 'v6'
language_code = 'en'
format_ = 'json'
api_key = 'qumcxrf65fvgnry82qnxhcxv'
year = 2018
nfl_season = 'REG'

sum_url = f"https://api.sportradar.us/nfl/official/{access_level}/{version}/{language_code}/games/{year}/{nfl_season}/schedule.{format_}?api_key={api_key}"
# summary = requests.get(sum_url).json()
# time.sleep(2)

In [55]:
# for cur_year in np.arange(2010,2016):
#     print(f'YEAR: {cur_year}')
    
#     # season_schedule
#     access_level = 'trial'
#     version = 'v6'
#     language_code = 'en'
#     format_ = 'json'
#     api_key = 'qumcxrf65fvgnry82qnxhcxv'
#     year = cur_year
#     nfl_season = 'REG'

#     sum_url = f"https://api.sportradar.us/nfl/official/{access_level}/{version}/{language_code}/games/{year}/{nfl_season}/schedule.{format_}?api_key={api_key}"
#     summary = requests.get(sum_url).json()
#     time.sleep(2)
    
#     all_ids = []
#     for week in range(len(summary['weeks'])):
#         cur_week = summary['weeks'][week].copy()

#         for game in range(len(cur_week['games'])):
#             all_ids.append(cur_week['games'][game]['id'])


#     with open(f'../data/sportsradar/game_ids/NFL/ids_{cur_year}.txt', 'w') as f:
#         for item in all_ids:
#             f.write("%s\n" % item)

In [659]:
# first go
# stopped @ 2016
# for cur_year in np.arange(2010,2020)[::-1]:
#     print(f'YEAR: {cur_year}')
    
#     # season_schedule
#     access_level = 'trial'
#     version = 'v6'
#     language_code = 'en'
#     format_ = 'json'
#     api_key = 'qumcxrf65fvgnry82qnxhcxv'
#     year = cur_year
#     nfl_season = 'REG'

#     sum_url = f"https://api.sportradar.us/nfl/official/{access_level}/{version}/{language_code}/games/{year}/{nfl_season}/schedule.{format_}?api_key={api_key}"
#     summary = requests.get(sum_url).json()
#     time.sleep(2)
    
#     all_ids = []
#     for week in range(len(summary['weeks'])):
#         cur_week = summary['weeks'][week].copy()

#         for game in range(len(cur_week['games'])):
#             all_ids.append(cur_week['games'][game]['id'])


#     with open(f'../data/sportsradar/game_ids/NFL/ids_{cur_year}.txt', 'w') as f:
#         for item in all_ids:
#             f.write("%s\n" % item)

#     pandas_list = []
#     for i, ids in enumerate(all_ids):
#         print(f'{i} game_id: {ids}')
#         pbp_url = f"https://api.sportradar.us/nfl/official/{access_level}/{version}/{language_code}/games/{ids}/pbp.{format_}?api_key={api_key}"
#         try:
#             pbp = requests.get(pbp_url).json()
#             joblib.dump(pbp, f'../data/sportsradar/pbp_json/pbp_{cur_year}_{ids}_.pkl')
#         except:
#             print(f'FAILED \n game_id:{ids}')
#             continue
    

YEAR: 2019
0 game_id: 06d50f40-10ca-4918-bda7-7df773a771f0
1 game_id: 26c758eb-ebf1-46aa-9fa8-6934a8a74b23
2 game_id: 31c88647-6e15-47c5-b196-e03a7991a75b
3 game_id: 3242f9ee-37c7-4508-aac9-6fb98259d673
4 game_id: 388f432f-f522-4552-9d81-55d64e804066
5 game_id: 3d3c2bda-e7b9-409a-9702-7ebaa0c7b5a3
6 game_id: 4b2875df-6f6b-4b33-9736-63e8637dcfef
7 game_id: 75dce1db-65cb-42a7-a8ed-7bf5e9142f69
8 game_id: 97af9f1a-cdae-40de-accc-080cb0d108bd
9 game_id: 9b261925-6913-4027-8348-2f93cc340956
10 game_id: ac9c23dc-8628-4954-84d1-26818cb0ac74
11 game_id: af36ccfd-98ee-4c93-827e-7384d67919f5
12 game_id: b71eb733-026a-49d1-a1a1-2d59893a2e44
13 game_id: d40d7cb4-2b6b-4f44-b540-dfbaf0582456
14 game_id: ebe4b2b3-cad3-42ca-9282-a8f44482d9a6
15 game_id: f5d68421-c3a5-4c69-8617-a9ebe8a5c997
16 game_id: 1dc044c4-a097-4dad-ae70-e82563602a9a
17 game_id: 271203ab-56dc-4fc8-906d-4835c79694cf
18 game_id: 2f490fd3-c6b3-4e0b-bc92-11ce30a4c1d8
19 game_id: 3224dbdf-ad1e-4389-b1b8-7ab57b7990fc
20 game_id: 3d3fa2e

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [656]:
# what I need to do is pull all the pbp data in json form first
# then I can do the cleaning after it is all local

#     pandas_list = []
#     for ids in all_ids:
#         print(ids)
        
#         pbp_url = f"https://api.sportradar.us/nfl/official/{access_level}/{version}/{language_code}/games/{ids}/pbp.{format_}?api_key={api_key}"
#         pbp = requests.get(pbp_url).json()
#         joblib.dump(pbp, f'../data/sportsradar/pbp_json_{cur_year}_{ids}_.pkl')
    

#         look = pbp.copy()
#         out = flat_pbp(look)
#         df_clean = clean_pbp(out)
  
#         joblib.dump(df_clean, f'../data/sportsradar/pbp_cleandf__{cur_year}_{ids}_.pkl')
        
#         pandas_list.append(df_clean)
        
#     joblib.dump(pd.concat(pandas_list), f'../data/sportsradar/pbp_df_{cur_year}_.pkl')

{'id': '5a5b0a83-1765-4729-b914-73ff7d4c4c89',
 'status': 'closed',
 'reference': '58172',
 'number': 5,
 'scheduled': '2020-09-13T17:00:00+00:00',
 'attendance': 0,
 'utc_offset': -5,
 'entry_mode': 'INGEST',
 'weather': 'Mostly Sunny Temp: 81 F, Humidity: 59%, Wind: N 5 mph',
 'clock': '00:00',
 'quarter': 4,
 'sr_id': 'sr:match:21980367',
 'summary': {'season': {'id': '2eec467f-9ce1-484c-b58f-e11615bede68',
   'year': 2020,
   'type': 'REG',
   'name': 'REG'},
  'week': {'id': 'fd51f745-b7eb-4c01-8550-d9095aa5407d',
   'sequence': 1,
   'title': '1'},
  'venue': {'id': '39be9ed4-3292-49ac-8699-a381de3a4969',
   'name': 'Bank of America Stadium',
   'city': 'Charlotte',
   'state': 'NC',
   'country': 'USA',
   'zip': '28202',
   'address': '800 South Mint Street',
   'capacity': 75419,
   'surface': 'artificial',
   'roof_type': 'outdoor',
   'sr_id': 'sr:venue:8173',
   'location': {'lat': '35.225937', 'lng': '-80.853133'}},
  'home': {'id': 'f14bf5cc-9a82-4a38-bc15-d39f75ed5314',


In [625]:
#play by play

# access_level = 'trial'
# version = 'v6'
# language_code = 'en'
# game_id = '0052ee6b-cf3e-4618-b30f-addbd4f32420'
# format_ = 'json'
# api_key = 'qumcxrf65fvgnry82qnxhcxv'

# url = f"https://api.sportradar.us/nfl/official/{access_level}/{version}/{language_code}/games/{game_id}/pbp.{format_}?api_key={api_key}"

In [626]:
# pbp = requests.get(url).json()

In [ ]:
# Start with simple
# slowly add complex


"""
simple play by play dataframe:
gameid playid hometeam awayteam 
posession team
quarter minute timeleft 
yardline hash down distance
formation run pass
run direction pass direction
"""

"""
advanced pbp:
current points
play count
drive number
season total situation play percentages
coaching career situation play percentages
playtype situation 1st down convergence percentages
star player influences
qb ratings
rb ratings
oline ratings
win record
league standings
rushing averages
rushing direction averages
playtype averages
"""


## NCAAFB

In [ ]:
# sports radad mens football sucks, no play direction

In [54]:
## to retrieve gameids

for cur_year in np.arange(2018,2021):
    print(f'YEAR: {cur_year}')

    version = 'v7'
    language_code = 'en'
    format_ = 'json'
    api_key = 'ch4eqj94ajgbqjtr6nj7g4nf'
    ncaafb_season = 'REG'
    

    url_base = f"https://api.sportradar.us/ncaafb/trial/v7/{language_code}/games/{cur_year}/{ncaafb_season}/schedule.{format_}?api_key={api_key}"
    summary = requests.get(url_base)
    time.sleep(5)
    
    print(f'status code: {summary.status_code}')
    if summary.status_code == 404:
        continue
        
    summary = summary.json()
    all_ids = []
    for week in range(len(summary['weeks'])):
        cur_week = summary['weeks'][week].copy()

        for game in range(len(cur_week['games'])):
            all_ids.append(cur_week['games'][game]['id'])


    with open(f'../data/sportsradar/game_ids/CFB/ids_{cur_year}.txt', 'w') as f:
        for item in all_ids:
            f.write("%s\n" % item)

YEAR: 2018
status code: 200
YEAR: 2019
status code: 200
YEAR: 2020
status code: 200


In [14]:
import os
import joblib
import requests
os.getcwd()

all_ids = open('../data/sportsradar/game_ids/CFB/ids_2014.txt').read().splitlines()

In [ ]:


url_base = f'https://api.sportradar.us/ncaafb/{access_level}/{version}/{language_code}/games/{game_id}/pbp.{format_}?api_key={your_api_key}'

pbp = requests.get(url_base).json()

In [15]:
access_level = 'trial'
version = 'v7'
language_code = 'en'
format_ = 'json'
api_key = 'ch4eqj94ajgbqjtr6nj7g4nf'
cur_year = 2014

i = 0
ids = all_ids[0]

# for i, ids in enumerate(all_ids):
print(f'{i} game_id: {ids}')
pbp_url = f"https://api.sportradar.us/ncaafb/{access_level}/{version}/{language_code}/games/{ids}/pbp.{format_}?api_key={api_key}"

pbp = requests.get(pbp_url).json()
joblib.dump(pbp, f'../data/sportsradar/pbp_json/CFB/pbp_{cur_year}_{ids}_.pkl')
#     if i == 0:
#         break
        


0 game_id: 02ff8b12-a52c-44a8-a83a-158034f36edd


['../data/sportsradar/pbp_json/CFB/pbp_2014_02ff8b12-a52c-44a8-a83a-158034f36edd_.pkl']

In [16]:
pbp = joblib.load('../data/sportsradar/pbp_json/CFB/pbp_2014_02ff8b12-a52c-44a8-a83a-158034f36edd_.pkl')

In [19]:
df = clean_pbp(flat_pbp(pbp))

KeyError: 'weather'

In [24]:
with open('test.txt','w') as f:
    f.write(str(pbp))